#### Testing out Gibbs zig-zag code:

In [1]:
using Distributions, TimeIt, ProgressMeter, PyPlot

In [2]:
include("zz_structures.jl")
include("mbsampler.jl")

get_clustering (generic function with 1 method)

In [3]:
d, Nobs = 15, 100
p = 1e-1
X = rand(Normal(), d, Nobs) .* rand(Binomial(1,p), d, Nobs)
X[1,:] = ones(Nobs)
ξ_true = rand(Normal(),d).* rand(Binomial(1,0.5), d)
ξ_true[1] = rand(Normal())
y = [rand(Binomial(1, 1/(1+exp(-ξ_true'X[:,j]))), 1)[1] + 0. for j in 1:Nobs];
mb_size = 10
σ02 = 1

# prior = HS_prior(d, σ02)
# prior = GDP_prior(d, σ02)
prior = gaussian_prior_nh(zeros(d), 10*rand(d))

gaussian_prior_nh([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.786996, 0.944773, 3.98202, 2.4836, 4.05132, 6.93516, 0.964916, 5.46668, 1.5502, 0.979238, 9.36217, 1.35441, 0.762928, 9.83863, 3.80299])

In [4]:
my_ll = ll_logistic(X,y);
my_model = model(my_ll, prior)
root = find_root(my_model, rand(d));

In [22]:
A = eye(d)
opf = projopf(A, 100, hyperparam_size(prior))
opf.alpha = 10*rand(d)
opt = maxa_opt(10^6)
outp = outputscheduler(opf,opt);

In [23]:
bb = linear_bound(my_model.ll, my_model.pr, gs_list, zz_state(opf))
update_bound(bb, my_ll, prior, gs_list, zz_state(opf));

In [24]:
print("Speed = ", opf.alpha)

Speed = [2.69215, 4.89587, 0.803515, 1.37942, 6.17534, 0.13602, 2.62024, 7.48714, 2.04688, 8.96947, 9.84737, 4.1929, 4.74923, 7.81902, 4.49522]

In [25]:
my_zz_sampler = zz_sampler(0, gs_list, bb)
hyper_sampler = block_gibbs_sampler(1.)
blocksampler = Array{msampler}(2)
blocksampler[1] = my_zz_sampler
blocksampler[2] = hyper_sampler;

In [26]:
start = time()
ZZ_block_sample(my_model::model, outp::outputscheduler, blocksampler::Array{msampler})
print( round((time()-start)/60, 2), " mins")

2.13 mins

In [19]:
n_samples = 10^4
xi_samples = extract_samples(outp.opf.xi_skeleton, 
                             outp.opf.bt_skeleton, 
                             outp.opf.bt_skeleton[end]/n_samples,
                             "linear");
hyper_samples = extract_samples(outp.opf.hyper_skeleton, 
                                outp.opf.bt_skeleton, 
                                outp.opf.bt_skeleton[end]/n_samples, 
                                "constant");

In [20]:
[compute_configT(my_model, xi_samples[:,1:end-1], hyper_samples[:,1:end-1], k) for k in 1:d]

15-element Array{Float64,1}:
 0.976321
 0.982059
 1.00033 
 1.05302 
 0.98071 
 0.95288 
 0.984481
 0.907669
 0.995672
 0.978412
 1.01546 
 1.02871 
 0.968577
 1.01633 
 1.16309 